In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import re

evalTime = 25.

In [2]:
def getVal(mypath, filename, header="infer",names = None):
    fullpath = pathresults +mypath+filename
    if names is None:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header).dropna(how= "all", axis=1)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names).dropna(how= "all", axis=1)
    return cst

def getTimeLine(path2file, evalTime):
    time = pd.read_csv(pathresults + path2file + "time.txt", names = ["time","Qlight"])["time"][1:] # because we have twice the initial value
    time_diff = abs(np.array(time)-evalTime)
    timeLineNeg = len(time_diff) - np.where(time_diff == min(time_diff))[0][0] 
    timeLinePos = len(time_diff) -timeLineNeg
    print('timeLine',timeLineNeg,timeLinePos )
    return timeLineNeg, timeLinePos
    
def get_last_line_as_numpy_array(file_path, timeLine,pos = False, dtype = float, verbose = False):
    with open(pathresults +file_path, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line
        
        try:
            if pos:
                last_line = lines[timeLine-1].strip()
            else:
                last_line = lines[-timeLine-1].strip()
        except:
            print('timeLine',file_path, len(lines)-timeLine, len(lines))
            raise Exception

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)
        if verbose:
            print(file_path, len(array_from_last_line), end=", ")

        return array_from_last_line

def getCylIdx(path2file, timeLinePos):
    fileName = "rhizoSegsId.txt"
    fullpath = pathresults +path2file+fileName
    rhizoSegsId = get_last_line_as_numpy_array(fullpath, timeLinePos,pos=True, dtype = int, verbose = True)
    print('rhizoSegsId',max(rhizoSegsId))
    return rhizoSegsId
    
def list_files_with_prefix(folder_path, prefix):
    file_names = []
    for file_name in os.listdir(pathresults+folder_path):
        if file_name.startswith(prefix):
            file_names.append(file_name)
    return file_names

# cylinder max konz (per cell)
def getData_(cid,gId,path2file, timeLineNeg):
    fileName = "cyl_val/Cyl_cellVol_"+str(gId)+".txt"
    cVol = get_last_line_as_numpy_array(path2file+fileName, timeLineNeg,pos= False, dtype = float)
    if cid == volIdx:
        return sum(cVol)
    elif cid <= 9:
        if cid <= 2:# == 0:
            fileName = "cyl_val/Cyl_watercontent_"+str(gId)+".txt"
            theta = get_last_line_as_numpy_array(path2file+fileName, timeLineNeg,pos= False,  dtype = float)
            cVol *= theta#cm3 scv to cm3 water
            if cid == 0:
                return sum(cVol)
        fileName = "cyl_val/Cyl_content"+str(cid)+"_"+str(gId)+".txt"
        Q1 = get_last_line_as_numpy_array(path2file+fileName, timeLineNeg, pos= False, dtype = float)
        konz = sum(Q1)/sum(cVol) #mol/cm3 or cm3/cm3
        return konz
    raise Exception

nKonz = 9
dictXYZ = {9:'X',10:'Y',11:'Z'}
volIdx=12
lenIdx=13
stIdx=14
orgIdId=15
relLenId=16

nToGet = relLenId+1 #elements + 3d coordinates of y , node+vol+len+st 

exceptPset =[]
scenarios = ["earlyDry","baseline","lateDry"]
numPset = len(scenarios)
allPSets = [ i for i in range(numPset) if i not in exceptPset]

def getData():
    
    GiniAll = [[[] for i in range(numPset)] for ii in range(nToGet)]
    
    for pSet in allPSets:
        print(pSet,end =", ")
        if True:

        #try:
            #path2file = 'none_55_'+str(pSet)+'_10to11_20mn_0s_5/'
            path2file = "forEGU"+scenarios[pSet]+"_1440_76_10to25_20mn_0s_128/"
            # path2files[pSet] #'1d1dnone_1440_'+str(pSet)+'_10to11_20mn_0s_5/'

            timeLineNeg, timeLinePos = getTimeLine(path2file, evalTime)
            
            
            rr = getCylIdx(path2file, timeLinePos)
            
            rr.sort()
            print('(',len(rr),')',end =", ")

            for cid in range(nKonz):
                Ginits = np.array([getData_( cid, gId,path2file, timeLineNeg) for gId in rr])          
                GiniAll[cid][pSet] = Ginits
            for cid in range(nKonz,volIdx):
                nodes_ = get_last_line_as_numpy_array(path2file+"nodes_"+dictXYZ[cid]+".txt", timeLinePos,pos = True, dtype =float)
                GiniAll[cid][pSet] = nodes_[rr+1]#seg idx to y-idx
            #cid = nToGet -1
            Ginits = np.array([getData_( volIdx, gId,path2file, timeLineNeg) for gId in rr])
            GiniAll[volIdx][pSet] = Ginits
            lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", timeLinePos,pos = True, dtype =float, verbose = True)
            GiniAll[lenIdx][pSet] = lens[rr]
            sts = get_last_line_as_numpy_array(path2file+"subTypes"+".txt", timeLinePos,pos = True, dtype =float, verbose = True)
            assert len(sts) == len(lens)
            GiniAll[stIdx][pSet] = sts[rr]

            orgidPerNode = get_last_line_as_numpy_array(path2file+"orgidPerNode.txt", timeLinePos,pos = True, dtype =int, verbose = True)
            globalNodeId = get_last_line_as_numpy_array(path2file+"globalNodeId.txt", timeLinePos,pos = True, dtype =int, verbose = True)
            parentNidx = get_last_line_as_numpy_array(path2file+"parentNidx.txt", timeLinePos, pos = True,dtype =int, verbose = True)
            ot_orgs = get_last_line_as_numpy_array(path2file+"ot_orgs.txt", timeLinePos,pos = True, dtype =int, verbose = True)
            id_orgs = get_last_line_as_numpy_array(path2file+"id_orgs.txt", timeLinePos, pos = True,dtype =int, verbose = True)
            id_roots = id_orgs[np.where(ot_orgs==2)]

            org2basenode = dict(zip(id_orgs, parentNidx))
            #org2ot = dict(zip(id_orgs, ot_orgs))
            isRoot = np.array([oo in id_roots for oo in orgidPerNode])

            orgidPerRoot = orgidPerNode[np.where(isRoot)]
            globalNodeIdroot =  globalNodeId[np.where(isRoot)]        
            segIdxRoot = np.array([rr for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]]) -1
            orgIdxRoot = np.array([orgidPerRoot[_] for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]])
            #segIdx2ot = dict(zip(segIdxRoot,orgIdxRoot))

            relLens = np.zeros(len(segIdxRoot))
            #lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
            for myorg in orgIdxRoot:
                thensegs = np.where(orgIdxRoot == myorg)#segIdxRoot[] 
                lenSegs = GiniAll[lenIdx][pSet]
                relLens[thensegs] = np.cumsum(lenSegs[thensegs])


            GiniAll[orgIdId][pSet] = orgIdxRoot
            GiniAll[relLenId][pSet] = relLens
        #except:
        #    print('jump',end =", ")

    return GiniAll #cid pSet rr
    

In [3]:

GiniAll = getData()

0, timeLine 1 1079
../results/forEGUearlyDry_1440_76_10to25_20mn_0s_128/rhizoSegsId.txt 1296, rhizoSegsId 1374
( 1296 ), forEGUearlyDry_1440_76_10to25_20mn_0s_128/length_Segs.txt 1375, forEGUearlyDry_1440_76_10to25_20mn_0s_128/subTypes.txt 1375, forEGUearlyDry_1440_76_10to25_20mn_0s_128/orgidPerNode.txt 1375, forEGUearlyDry_1440_76_10to25_20mn_0s_128/globalNodeId.txt 1375, forEGUearlyDry_1440_76_10to25_20mn_0s_128/parentNidx.txt 650, forEGUearlyDry_1440_76_10to25_20mn_0s_128/ot_orgs.txt 650, forEGUearlyDry_1440_76_10to25_20mn_0s_128/id_orgs.txt 650, 1, timeLine 1 1079
../results/forEGUbaseline_1440_76_10to25_20mn_0s_128/rhizoSegsId.txt 1725, rhizoSegsId 1812
( 1725 ), forEGUbaseline_1440_76_10to25_20mn_0s_128/length_Segs.txt 1813, forEGUbaseline_1440_76_10to25_20mn_0s_128/subTypes.txt 1813, forEGUbaseline_1440_76_10to25_20mn_0s_128/orgidPerNode.txt 1813, forEGUbaseline_1440_76_10to25_20mn_0s_128/globalNodeId.txt 1813, forEGUbaseline_1440_76_10to25_20mn_0s_128/parentNidx.txt 840, forEGU

In [76]:
np.array(GiniAll).shape

/tmp/ipykernel_1938202/2873905747.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(GiniAll).shape


(17, 2)

In [4]:
# get param set
outputvalsname =  ['wat','cs','cl','coa','cod','cca','ccd','css2','co2','yX','yY','yZ','vol','lenSeg','st','orgId','relLen']
assert len(outputvalsname) == nToGet 
column_names = outputvalsname + ['scenario']
df = pd.DataFrame(columns=column_names)

for pset in allPSets:#range(numPset):
    N = GiniAll[0][pset].shape[0]
    #assert N == 232
    rows_df = pd.DataFrame()
    
    for jj, outVn in enumerate(outputvalsname):
        rows_df[outVn] = GiniAll[jj][pset]
        #print('adding',jj,outVn,duplicated_rows_df.shape,len(GiniAll[jj][pset]))
    rows_df['scenario'] = scenarios[pset]
    
    df = pd.concat([df, rows_df], ignore_index=True)
df['co'] = df['coa']+df['cod']
df['cc'] = df['cca']+df['ccd']

In [79]:
df

,wat,cs,cl,coa,cod,cca,ccd,css2,co2,yX,yY,yZ,vol,lenSeg,st,orgId,relLen,scenario,co,cc
0,0.047567,2.398120e-06,0.000702,0.000002,0.000008,0.000071,0.000083,0.000055,0.008122,-0.298434,-0.304855,-1.204434,0.142857,1.0,0.0,1,1.000000,earlyDry,0.000010,0.000155
1,0.008591,2.342705e-06,0.000839,0.000002,0.000007,0.000131,0.000207,0.000052,0.018311,0.043922,-0.540903,-2.113870,0.025478,1.0,1.0,1,2.000000,earlyDry,0.000010,0.000337
2,0.010808,2.491620e-06,0.000782,0.000002,0.000008,0.000106,0.000138,0.000056,0.013091,-0.447197,-0.693829,-2.971433,0.031848,1.0,1.0,1,3.000000,earlyDry,0.000010,0.000244
3,0.008834,2.704320e-06,0.000801,0.000002,0.000007,0.000124,0.000147,0.000062,0.014621,-0.104340,-0.792904,-3.905582,0.025834,1.0,1.0,1,4.000000,earlyDry,0.000010,0.000271
4,0.008824,2.965022e-06,0.000814,0.000002,0.000007,0.000129,0.000175,0.000067,0.016487,-0.125120,-0.885239,-4.901093,0.025570,1.0,1.0,1,5.000000,earlyDry,0.000010,0.000305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,0.017915,1.221056e-06,0.000677,0.000002,0.000008,0.000015,0.000042,0.000077,0.002658,-0.345252,-0.094957,-4.972233,0.051924,1.0,1.0,7,3.024341,baseline,0.000010,0.000057
1949,0.016424,8.716080e-07,0.000689,0.000002,0.000007,0.000021,0.000056,0.000073,0.003801,-0.231014,0.601599,-4.171595,0.048037,1.0,1.0,7,4.024341,baseline,0.000010,0.000077
1950,0.024839,1.459167e-06,0.000655,0.000002,0.000008,0.000008,0.000021,0.000065,0.001099,0.620673,-1.179878,-16.387318,0.064197,1.0,1.0,7,5.024341,baseline,0.000009,0.000029
1951,0.045995,2.098200e-06,0.000652,0.000001,0.000008,0.000003,0.000014,0.000062,0.000414,-0.213043,3.285480,-15.276140,0.121212,1.0,1.0,7,6.024341,baseline,0.000009,0.000017


In [5]:
df.to_pickle('./forEgu25.pkl')